In [28]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI
from langchain_core.tools import Tool
from crewai_tools import tool, BaseTool,FileReadTool
from tempfile import TemporaryDirectory

from langchain_community.agent_toolkits import FileManagementToolkit
from langchain_community.utilities import GoogleSerperAPIWrapper

from langchain_google_community import GoogleSearchAPIWrapper

import os
import requests

In [29]:
os.environ["OPENAI_API_KEY"] = ""
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'

os.environ["GOOGLE_CSE_ID"] = ""
os.environ["GOOGLE_API_KEY"] = ""


os.environ["SERPER_API_KEY"] = ""

In [30]:
llm = ChatOpenAI(
    model = "crewai-llama2",
    base_url = "http://localhost:11434/v1")

In [31]:
# Tools

human_tool = load_tools(["human"])

# Google Search

search = GoogleSerperAPIWrapper()


# google_search_tool = Tool(
#     name = "google_search",
#     description = "Used to search information. Enter a query: ",
#     func = search.run,
# )

#search = GoogleSearchAPIWrapper()


google_search_tool = Tool(
    name="google_search",
    description="Search Google for recent results. Action Input: {{query: ""}}",
    func=search.run,
)


# Image Creation Tool
image_tool = load_tools(["dalle-image-generator"])

@tool("image_download_tool")
def image_download_tool(url: str, file_name: str) -> str:
    """Used to download images from a url to a file. Give the url and the name the file should download as with the file ending."""

    response = requests.get(url)
    with open(f"{file_name}", "wb") as file:
    # Write the content of the response (image) to the file
        file.write(response.content)

    return f"File downloaded as {file_name}"

# File System Tools

file_tool = [FileReadTool()]

In [32]:
# Unused Tools
#search = GoogleSerperAPIWrapper(type = 'image')
#search = GoogleSearchAPIWrapper(type = 'image')
# google_images_tool = Tool(
#     name = "google_search_images",
#     description = "Search Google for images needed for {topic}. ",
#     func = search_images.run,
# )

# Website Requests tool
request_tool = load_tools(["requests_get"], allow_dangerous_tools = True)

In [33]:
# Manager Agents

planning_manager = Agent(
  name = "planning_manager",
  #llm = llm,
  role='planning_manager',
  goal = (
    'Use the file_tool to read the neccessary game mechanics and images from the researcher_task. '
    'Plan how to create {topic}. '
    'Organize the tasks that must be completed. '
    #'Only delegate the task of reviewing your work to the reviewer. '
    #'Delegate these tasks to co-workers. '
    #"Use the researcher to research and plan. "
    #'Use the creative manager to oversee the artistic development of {topic}'
    "When using the Action: Delegate work to co-worker always specify the coworker's role. "
    #"When using the Action: Ask question to co-worker always specify the coworker's role. "
  ),
  verbose = True,
  memory = True,
  backstory = (
    "Dedicated manager who organizes tasks and delegates them effectively. "
  ),
  tools = file_tool,
  allow_delegation = True,
)

coding_manager = Agent(
  name = "coding_manager",
  #llm = llm,
  role='coding_manager',
  goal = (
    'Use the file_tool to read the coding plan from the task_plans. '
    'Over see the coding of {topic}. '
    'Organize the tasks that must be completed. '
    'Delegate the task of coding {topic} to the programmer. '
    'Delegate the task of testing the code to the code tester. '
    #"Use the researcher to research and plan. "
    #'Use the creative manager to oversee the artistic development of {topic}'
    "When using the Action: Delegate work to co-worker always specify the coworker's role. "
    "When using the Action: Ask question to co-worker always specify the coworker's role. "
  ),
  verbose = True,
  memory = True,
  backstory = (
    "Dedicated manager who organizes tasks and delegates them effectively. "
  ),
  tools = file_tool,
  allow_delegation = True,
)

In [34]:
# Research Agent

researcher = Agent(
    name = "researcher",
    role = "researcher",
    #llm = llm,
    goal = (
        "Informer your co-workers on the game mechanics and images needed to create {topic} with {topic_info} "
        "When using the Action: google_searh use the format Action Input: {{query: ""}}. "
        "Use the google_search_tool to learn about {topic}. "
        # "Follow tasks given to by co-workers. "
        # "Answer questions and supply information to co-workers. "
      
    ),
    verbose = True,
    memory = True,
    backstory = (
        "Extremely knowledgeable about video games, and video game development. "
        "Skilled user of google search. "
    ),
    tools = [google_search_tool],
    allow_delegation = False,
)

In [35]:
# Artist Agent

artist = Agent(
  name = "artist",
  #llm = llm,
  role = "artist",
  goal = (
    #'Search Google for files to download. '
    #'Download files from urls found from searching google. '
    "Generate, and download a maximum of 5 images. "
    'Generate images requested by the programmer. '
    'Download all images and name them short descriptive names. '
    'Make sure your images are scalled properly. '                                        #Topic_Specifications
    'Always include file endings in image names.'
    'Report to the programmer the name the image has been downloaded as. '
    'All images downloaded must be of single things. Do not ask for multiple images in your prompts. '
  ),
  verbose = True,
  memory = True,
  backstory = (
    "Skilled dalle image generator prompter. "
  ),
  tools = [image_tool[0], image_download_tool, file_tool[0]], #[image_tool[0], google_images_tool, request_tool[0], download_tool],
  allow_delegation = False
)

In [36]:
# Coding Agents
programmer = Agent(
  name = "programmer",
  #llm = llm,
  role='programmer',
  goal= (
    #f'Use the file_tool to read the task instructions from the disk with file path {planning_task.output_file}. '
    'Write perfect python code. '
    'Your finale answer must be a single python.'
    'Resolve any bugs or errors in the code. '
    "Iteratively code, test for bugs and errors, and update that code. "
    "Delegate the creation of images to the artist. "
    "Use the file names the artist gives you. "
    #f'Make sure that the images are named the same as in {art_task.output_file}. '
    #'Make sure your code includes {topic_info}'
    #'Make sure your code scalles images properly'                                                               #Specifications
    "When using the Action: Ask question to co-worker always specify the coworker: round_one_tester. "
    "When using the Action: Delegate work to co-worker always specify the coworker's role. "
  ),
  verbose=True,
  memory=True,
  backstory=(
    "Experienced in coding and debugging.  "
  ),
  allow_delegation = True
)

# Creating a writer agent with custom tools and delegation capability
code_tester = Agent(
  name = "code_tester",
  #llm = llm,
  role='code_tester',
  goal= (
      "Assist the programmer in developing code. "
      "Test code to find and report bugs and errors. "
      "Make sure that the images are correctly named in the code. "

    # 'Give feedback on {topic}, and ensure that the round_one_programmer knows of any buggs or errors. '
    # 'Work with round_one_bug_fixer. '
    # 'Work before the round two and round three co-workers. '
  ),
  verbose=False,
  memory=True,
  backstory=(
    "Experienced code tester. "
  ),
  allow_delegation = True
)

In [37]:
#Human Feedbacker Agent

human_feedbacker = Agent(
    name = "human_feedbacker",
    role = "human_feedbacker",
    #llm = llm,
    goal = (
        "Do not finalize your answer until you recieve feedback that the code is complete. "
        "Ask the human for feedback on the creation of {topic}. "
        "Iteratively ask the human for feedback then delegate the task of coding that feedback to the programmer. "
        "Use the file_tool to access the current code from the disk with file path game.py "
        "Delegate the task of coding to the programmer. "
        "Allways give the codder the current code when delegating work. "
        "When using the Action: Delegate work to co-worker always specify the coworker's role. "
    ),
    backstory = (
        "Expirenced at game development. "
    ),
    verbose = True,
    memory = True,
    tools = human_tool,
    allow_delegation = True
)

In [38]:
# Researcher Task
researcher_task = Task (
    description = (
        "Allways give an answer. "
        "Use google_search_tool to research what game mechanics are. "
        "Use the google_search_tool to research {topic}. "
        "Find the necessary game features and images for {topic}. "
        "Make sure these game features and images include {topic_info}. "
        "Do not generate or source any images in this task. "
        "Do not code in this task. "
    ),
    expected_output = (
        "A listof the necessary game features and a seperate list of images for {topic}. "
    ),
    tools = [google_search_tool],
    agent = researcher,
    human_input = False,
    async_execution = False,
    output_file = "game_research.md"
)

In [39]:
# Planning Task
planning_task = Task(
    description = (
        f"Use the file_tool to read the neccesary game features and images from the disk with file path {researcher_task.output_file}"
        "Use the information gathered by the researcher during the research task. Do not do your own research, or delegate research. "
        "Generate instructions for the coding task. "
        #"Delegate the task of reviewing you work to the reviewer. "
        #'Only delegate the task of reviewing your work to the reviewer. '
        'Do not do any coding or generating images. '
        #"Do not finalize your answer until the reviewer confirms you have included all the neccessary images. "
        #"Research and plan the game features needed for {topic}. "
        #"Research and plan the images needed for {topic}. "
    ),
    expected_output = (
        "Detailed plans and instructions for the coding task. "
    ),
    tools = file_tool,
    agent = planning_manager,
    human_input = False,
    async_execution = False,
    output_file = "task_plans.md"
)

In [40]:
# Coding Task
coding_task = Task (
    description = (
        f"Use the file tool to read the task instructions from the disk with file path {planning_task.output_file}. "
        #f"use the file tool to read the information the disk with the file path {art_task.output_file} and use these names when naming images in the code. "
        "Create perfect python code to implement everything in the coding task. "
        #"Make sure that the images are correctly named in the code. "
        #"Make sure that the images are scalled correctly. "
        'Make sure the code includes {topic_info}'
        "Iteratively code, test for bugs and errors, and update that code. "
        "Your final answer should include python code for {topic} and only python code for {topic}. "
        'Use the artist to generate any images needed. '
        #"Delegate the task of reviewing you work to the reviewer. "
        #"Do not finalize your answer until the reviewer confirms you have included all the neccessary images. "
        #"Add features to the code to create a fully codded {topic}. "
        #"Ask the creative manager for assistance using the images if neccesary. "
    ),
    expected_output = (
        "A single executable file that contains python code for {topic}. "
    ),
    agent = coding_manager,
    human_input = False,
    async_execution = False,
    output_file = "game.py"
)

In [41]:
# Human Feedback Task
human_feedback_task = Task(
    description = (
        "Do not finalize your answer until the human confirms the code is complete. "
        "Use the human_tool to ask the human for feedback on the code. "
        "Iteratively ask the human for feedback then delegate the task of coding that feedback to the programmer. "
        "Do this until the human confirms the code is complete. "
        "Your final answer should include python code for {topic} and only python code for {topic}. "
    ),
    expected_output = (
        "A single executable file that contains python code for {topic}. "
    ),
    #tools = file_tool,
    agent = human_feedbacker,
    human_input = False,
    async_execution = False,
    output_file = "game.py",
)

In [42]:
manager_crew = Crew(
  #agents = [manager, creative_manager, researcher, artist, programmer, code_tester, game_tester],
  #tasks = [planning_task, art_task, coding_task, human_feedback_task, human_feedback_planning_task, art_task, coding_task],
  agents = [planning_manager, coding_manager, researcher, artist, programmer, code_tester, human_feedbacker],
  tasks = [researcher_task, planning_task, coding_task, human_feedback_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  #manager_llm = ChatOpenAI(model = 'gpt-3.5-turbo'),
  verbose = 2,
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True,
  step_callback = print("STEP ################################################"),
  max_iter = 5
)

2024-06-12 16:29:06,964 - 124474574772032 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


STEP ################################################


In [44]:
if os.path.exists('game.py'):
    os.remove("game.py")
if os.path.exists('game_research.md'):
    os.remove("game_research.md")
if os.path.exists('task_plans.md'):
    os.remove("task_plans.md")
if os.path.exists('art_information.md'):
    os.remove("art_information.md")
# Get the current directory
current_directory = os.getcwd()

# Loop through all files in the current directory
for filename in os.listdir(current_directory):
    # Check if the file ends with .png
    if filename.endswith(".png"):
        # Create the full file path
        file_path = os.path.join(current_directory, filename)
        # Delete the file
        os.remove(file_path)

    
result = manager_crew.kickoff(inputs={'topic': 'A 2d platformer game', 'topic_info': 'A dinosaur character, 5 images scalled properly, all images should be of single entities do not use sprite sheets, controls to make the character move left, right and jump and multiple platforms for the character to jump on.'})

print(result)
    

 [DEBUG]: == Working Agent: researcher
 [INFO]: == Starting Task: Allways give an answer. Use google_search_tool to research what game mechanics are. Use the google_search_tool to research A 2d platformer game. Find the necessary game features and images for A 2d platformer game. Make sure these game features and images include A dinosaur character, 5 images scalled properly, all images should be of single entities do not use sprite sheets, controls to make the character move left, right and jump and multiple platforms for the character to jump on.. Do not generate or source any images in this task. Do not code in this task. 


> Entering new CrewAgentExecutor chain...
To provide accurate and comprehensive information to my co-workers on the game mechanics and images needed to create a 2D platformer game with a dinosaur character, I will first gather information on game mechanics and 2D platformer games. Then, I will use this information to list the necessary game features and the spec

2024-06-12 17:13:56,891 - 124474574772032 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--4a1d2d67311c99c8821bc2da176341cdee81ead432cb58c008f2d91c73367d35


### Thought:
I need to ask the human for feedback on the updated code to ensure it meets all the requirements and if there are any additional changes or features needed.

### Action:
Ask the human for feedback on the updated code.

Action: human
Action Input: {"question": "The code has been updated to allow the dinosaur to land on platforms, add a background image, replace platform rectangles with images, add collectible items, and add a scoring system to count collected items. Can you provide further feedback? Are there any additional changes or features you would like to see?"}
 

Action 'Ask the human for feedback on the updated code.

Action: human' don't exist, these are the only available Actions:
 human - You can ask a human for guidance when you think you got stuck or you are not sure what to do next. The input should be a question for the human.
Delegate work to co-worker(task: str, context: str, coworker: Optional[str] = None, **kwargs) - Delegate a specific task to one of th